In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np
from sklearn.model_selection import train_test_split , StratifiedKFold , cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest , f_classif
from sklearn.metrics import accuracy_score

In [2]:
def make_cats_vs_dogs(seed = 42 , n_per_class = 80 , p_noise = 50000):
    rng = np.random.default_rng(seed)

    cats = rng.multivariate_normal(mean = [-0.2 , -0.2] , cov = [[1.3 , 0.2] , [0.2 , 1.3]] , size = n_per_class)
    dogs = rng.multivariate_normal(mean = [0.2 , 0.2] , cov = [[1.3 , -0.2] , [-0.2 , 1.3]] , size = n_per_class)

    X_base = np.vstack([cats , dogs])
    y = np.hstack([np.zeros(n_per_class) , np.ones(n_per_class)])

    noise = rng.normal(0 , 1 , size = (X_base.shape[0] , p_noise))
    X = np.hstack([X_base , noise])

    return X , y

In [3]:
def train_and_evaluate(X , y , k = 5 , seed = 42):

    X_train , X_tmp , y_train , y_tmp = train_test_split(
        X , y , test_size = 0.4 , random_state = seed , stratify = y
    )
    X_val , X_test , y_val , y_test = train_test_split(
        X_tmp , y_tmp , test_size = 0.5 , random_state = seed , stratify = y_tmp
    )

    clf = LogisticRegression(max_iter = 5000 , solver = "liblinear")

    #=======================================
    # Selector inside Pipeline (no leakage)
    #=======================================
    pipe = Pipeline([
        ("selector" , SelectKBest(score_func = f_classif , k = k)) ,
        ("scaler" , StandardScaler()) , ("clf" , clf)
    ])

    pipe.fit(X_train , y_train)
    val_acc = accuracy_score(y_val , pipe.predict(X_val))
    test_acc = accuracy_score(y_test , pipe.predict(X_test))

    #=======================
    # CV score on full data
    #=======================
    cv = StratifiedKFold(n_splits = 5 , shuffle = True , random_state = seed)
    cv_acc = cross_val_score(pipe , X , y , cv = cv , scoring = "accuracy").mean()

    print("Results (Accuracy): ")
    print("Val: " , round(val_acc , 3))
    print("Test: " , round(test_acc , 3))
    print("5-Fold CV (Pipeline): " , round(cv_acc , 3))

    #===================================
    # Quick leakage red-flag heuristic
    #===================================
    if val_acc - test_acc > 0.10:
        print("⚠️  Red flag: big Val→Test drop. Check leakage / split strategy / drift.")

In [4]:
#===============
# Run Template
#===============
X , y = make_cats_vs_dogs(seed = 42 , n_per_class = 80 , p_noise = 50000)
train_and_evaluate(X , y , k = 5 , seed = 42)

Results (Accuracy): 
Val:  0.5
Test:  0.562
5-Fold CV (Pipeline):  0.506
